In [63]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score

ModuleNotFoundError: No module named 'seaborn'

In [38]:
df = pd.read_csv('data/football_data.csv')


In [39]:
df.head()


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Season
0,17/08/13,Arsenal,Aston Villa,1.0,3.0,A,1.0,1.0,D,A Taylor,...,4.0,3.0,4.0,5.0,1.0,0.0,1.44,4.75,8.0,2013
1,17/08/13,Liverpool,Stoke,1.0,0.0,H,1.0,0.0,H,M Atkinson,...,12.0,6.0,1.0,1.0,0.0,0.0,1.40,5.00,9.5,2013
2,17/08/13,Norwich,Everton,2.0,2.0,D,0.0,0.0,D,M Oliver,...,6.0,8.0,2.0,0.0,0.0,0.0,3.20,3.40,2.4,2013
3,17/08/13,Sunderland,Fulham,0.0,1.0,A,0.0,0.0,D,N Swarbrick,...,6.0,1.0,0.0,3.0,0.0,0.0,2.30,3.40,3.4,2013
4,17/08/13,Swansea,Man United,1.0,4.0,A,0.0,2.0,A,P Dowd,...,7.0,4.0,1.0,3.0,0.0,0.0,4.20,3.50,2.0,2013


In [40]:
# Create Home Win, Draw Win and Away Win columns
df = df.assign(homeWin=lambda df: df.apply(lambda row: 1 if row.FTHG > row.FTAG else 0, axis='columns'),
              draw=lambda df: df.apply(lambda row: 1 if row.FTHG == row.FTAG else 0, axis='columns'),
              awayWin=lambda df: df.apply(lambda row: 1 if row.FTHG < row.FTAG else 0, axis='columns'))

In [41]:
# lower case all column names
df.columns = [x.lower() for x in df.columns]

In [42]:
df.columns

Index(['date', 'hometeam', 'awayteam', 'fthg', 'ftag', 'ftr', 'hthg', 'htag',
       'htr', 'referee', 'hs', 'as', 'hst', 'ast', 'hf', 'af', 'hc', 'ac',
       'hy', 'ay', 'hr', 'ar', 'b365h', 'b365d', 'b365a', 'season', 'homewin',
       'draw', 'awaywin'],
      dtype='object')

In [43]:
win_rates = \
(df[['homewin', 'draw', 'awaywin']]
    .astype(float)
    .groupby(df['season'])
    .mean())

win_rates

,homewin,draw,awaywin
season,,,
2013,0.471053,0.205263,0.323684
2014,0.451444,0.244094,0.301837
2015,0.413158,0.281579,0.305263
2016,0.492105,0.221053,0.286842
2017,0.455263,0.260526,0.284211
2018,0.476316,0.186842,0.336842
2019,0.452632,0.242105,0.305263
2020,0.378947,0.218421,0.402632
2021,0.428947,0.231579,0.339474


In [30]:
import matplotlib.pyplot as plt

# Set the style
plt.style.use('ggplot')

fig = plt.figure()
ax = fig.add_subplot(111)

home_line = ax.plot(win_rates.homeWin, label='Home Win Rate')
away_line = ax.plot(win_rates.awayWin, label='Away Win Rate')
draw_line = ax.plot(win_rates.draw, label='Draw Win Rate')
ax.set_xlabel("season")
ax.set_ylabel("Win Rate")
plt.title("Win Rates", fontsize=16)

# Add the legend locations
home_legend = plt.legend(handles=home_line, loc='upper right', bbox_to_anchor=(1, 1))
ax = plt.gca().add_artist(home_legend)
away_legend = plt.legend(handles=away_line, loc='center right', bbox_to_anchor=(0.95, 0.4))
ax = plt.gca().add_artist(away_legend)
draw_legend = plt.legend(handles=draw_line, loc='center right', bbox_to_anchor=(0.95, 0.06))

ModuleNotFoundError: No module named 'matplotlib'

In [47]:
home_win_rates = \
(df.groupby(['hometeam'])
    .homewin
    .mean())

away_win_rates = \
(df.groupby(['awayteam'])
    .awaywin
    .mean())

hga = (home_win_rates - away_win_rates).reset_index().rename(columns={0: 'hga'}).sort_values(by='hga', ascending=False)

In [48]:
hga.head(10)

,hometeam,hga
22,Nott'm Forest,0.339744
12,Hull,0.228070
24,Sheffield United,0.227273
0,Arsenal,0.213745
26,Stoke,0.210526
23,QPR,0.210526
29,Tottenham,0.184121
9,Everton,0.183673
15,Liverpool,0.173469
20,Newcastle,0.163842


In [55]:
big_clubs = ['Liverpool', 'Man City', 'Man United', 'Chelsea', 'Arsenal']
home_win_rates_5 = df[df['hometeam'].isin(big_clubs)].groupby(['hometeam', 'season'])['homewin'].mean()
away_win_rates_5 = df[df['awayteam'].isin(big_clubs)].groupby(['awayteam', 'season'])['awaywin'].mean()

hga_top_5 = home_win_rates_5 - away_win_rates_5

hga_top_5.unstack(level=0)

hometeam,Arsenal,Chelsea,Liverpool,Man City,Man United
season,,,,,
2013,0.105263,0.263158,0.315789,0.368421,-0.052632
2014,0.105263,0.210526,0.105263,0.210526,0.421053
2015,0.210526,-0.105263,0.000000,0.263158,0.263158
2016,0.263158,0.210526,0.105263,-0.052632,-0.105263
2017,0.578947,0.052632,0.157895,0.000000,0.263158
2018,0.368421,0.157895,0.210526,0.210526,0.052632
2019,0.315789,0.105263,0.210526,0.210526,0.105263
2020,-0.105263,-0.052632,0.000000,-0.052632,-0.157895
2021,0.210526,-0.157895,0.105263,0.052632,0.210526


In [56]:
sns.lineplot(x='season', y='hga', hue='team', data=hga_top_5.reset_index().rename(columns={0: 'hga', 'hometeam': 'team'}))
plt.legend(loc='lower center', ncol=6, bbox_to_anchor=(0.45, -0.2))
plt.title("HGA Among the top 5 clubs", fontsize=14)
plt.show()

NameError: name 'sns' is not defined

In [10]:
# some teams are relegated, not all team have all seasons
matches['hometeam'].value_counts()


hometeam
Arsenal             197
Man United          197
Chelsea             197
Everton             196
Liverpool           196
West Ham            196
Crystal Palace      196
Tottenham           195
Man City            195
Southampton         190
Newcastle           177
Leicester           171
Aston Villa         139
Burnley             139
Brighton            121
Bournemouth         121
West Brom           114
Watford             114
Wolves              101
Stoke                95
Swansea              95
Fulham               81
Norwich              76
Sunderland           76
Leeds                57
Hull                 57
Brentford            44
Sheffield United     44
Cardiff              38
Huddersfield         38
Nott'm Forest        24
Middlesbrough        19
QPR                  19
Luton                 5
Name: count, dtype: int64

In [7]:
# some maches are still not played
matches['round'].value_counts()


round
Matchweek 1     120
Matchweek 8     120
Matchweek 2     120
Matchweek 12    120
Matchweek 11    120
Matchweek 10    120
Matchweek 9     120
Matchweek 7     120
Matchweek 6     120
Matchweek 5     120
Matchweek 4     120
Matchweek 3     120
Matchweek 34    100
Matchweek 29    100
Matchweek 30    100
Matchweek 31    100
Matchweek 33    100
Matchweek 32    100
Matchweek 28    100
Matchweek 35    100
Matchweek 36    100
Matchweek 37    100
Matchweek 26    100
Matchweek 27    100
Matchweek 20    100
Matchweek 25    100
Matchweek 24    100
Matchweek 23    100
Matchweek 22    100
Matchweek 21    100
Matchweek 19    100
Matchweek 18    100
Matchweek 17    100
Matchweek 16    100
Matchweek 15    100
Matchweek 14    100
Matchweek 13    100
Matchweek 38    100
Name: count, dtype: int64

In [8]:
matches.dtypes


date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf              float64
ga              float64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
pk                int64
g/sot           float64
dist            float64
sot%            float64
npxg            float64
fk              float64
np:g-xg         float64
g-xg            float64
sot             float64
g/sh            float64
sh              float64
pkatt             int64
npxg/sh         float64
gls               int64
season            int64
team             object
dtype: object

In [9]:
# convert date to datetime
matches['date'] = pd.to_datetime(matches['date'])


In [10]:
matches.dtypes


date            datetime64[ns]
time                    object
comp                    object
round                   object
day                     object
venue                   object
result                  object
gf                     float64
ga                     float64
opponent                object
xg                     float64
xga                    float64
poss                   float64
attendance             float64
captain                 object
formation               object
referee                 object
match report            object
notes                  float64
pk                       int64
g/sot                  float64
dist                   float64
sot%                   float64
npxg                   float64
fk                     float64
np:g-xg                float64
g-xg                   float64
sot                    float64
g/sh                   float64
sh                     float64
pkatt                    int64
npxg/sh                float64
gls     

In [11]:
#converting to category and then to numbers
matches["venue_code"] = matches["venue"].astype('category').cat.codes


In [12]:
matches["venue_code"].value_counts()


venue_code
0    2020
1    2020
Name: count, dtype: int64

In [13]:
matches.sort_values(by=['date'], inplace=True)


In [14]:
matches.head()


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,g-xg,sot,g/sh,sh,pkatt,npxg/sh,gls,season,team,venue_code
3470,2018-08-10,20:00,Premier League,Matchweek 1,Fri,Home,W,2.0,1.0,Leicester City,...,0.5,5.0,0.14,7.0,1,0.10,2,2018,Manchester United,1
3584,2018-08-10,20:00,Premier League,Matchweek 1,Fri,Away,L,1.0,2.0,Manchester Utd,...,-0.8,4.0,0.08,13.0,0,0.14,1,2018,Leicester City,0
3356,2018-08-11,15:00,Premier League,Matchweek 1,Sat,Away,W,3.0,0.0,Huddersfield,...,1.1,3.0,0.17,12.0,1,0.10,3,2018,Chelsea,0
3964,2018-08-11,15:00,Premier League,Matchweek 1,Sat,Home,L,0.0,2.0,Crystal Palace,...,-0.7,6.0,0.00,15.0,0,0.05,0,2018,Fulham,1
3888,2018-08-11,15:00,Premier League,Matchweek 1,Sat,Away,L,0.0,2.0,Watford,...,-0.3,0.0,0.00,6.0,0,0.05,0,2018,Brighton and Hove Albion,0


In [15]:
#create a unique code for each opponent
matches['opp_code'] = matches['opponent'].astype('category').cat.codes


In [16]:
matches.head()


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,sot,g/sh,sh,pkatt,npxg/sh,gls,season,team,venue_code,opp_code
3470,2018-08-10,20:00,Premier League,Matchweek 1,Fri,Home,W,2.0,1.0,Leicester City,...,5.0,0.14,7.0,1,0.10,2,2018,Manchester United,1,13
3584,2018-08-10,20:00,Premier League,Matchweek 1,Fri,Away,L,1.0,2.0,Manchester Utd,...,4.0,0.08,13.0,0,0.14,1,2018,Leicester City,0,17
3356,2018-08-11,15:00,Premier League,Matchweek 1,Sat,Away,W,3.0,0.0,Huddersfield,...,3.0,0.17,12.0,1,0.10,3,2018,Chelsea,0,11
3964,2018-08-11,15:00,Premier League,Matchweek 1,Sat,Home,L,0.0,2.0,Crystal Palace,...,6.0,0.00,15.0,0,0.05,0,2018,Fulham,1,8
3888,2018-08-11,15:00,Premier League,Matchweek 1,Sat,Away,L,0.0,2.0,Watford,...,0.0,0.00,6.0,0,0.05,0,2018,Brighton and Hove Albion,0,24


In [17]:
# convert time to hour, withhou minutes
matches['hour'] = matches['time'].str.replace(":.+", "", regex=True).astype(int)


In [18]:
matches.head()


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,g/sh,sh,pkatt,npxg/sh,gls,season,team,venue_code,opp_code,hour
3470,2018-08-10,20:00,Premier League,Matchweek 1,Fri,Home,W,2.0,1.0,Leicester City,...,0.14,7.0,1,0.10,2,2018,Manchester United,1,13,20
3584,2018-08-10,20:00,Premier League,Matchweek 1,Fri,Away,L,1.0,2.0,Manchester Utd,...,0.08,13.0,0,0.14,1,2018,Leicester City,0,17,20
3356,2018-08-11,15:00,Premier League,Matchweek 1,Sat,Away,W,3.0,0.0,Huddersfield,...,0.17,12.0,1,0.10,3,2018,Chelsea,0,11,15
3964,2018-08-11,15:00,Premier League,Matchweek 1,Sat,Home,L,0.0,2.0,Crystal Palace,...,0.00,15.0,0,0.05,0,2018,Fulham,1,8,15
3888,2018-08-11,15:00,Premier League,Matchweek 1,Sat,Away,L,0.0,2.0,Watford,...,0.00,6.0,0,0.05,0,2018,Brighton and Hove Albion,0,24,15


In [19]:
#get week day
matches['day_code'] = matches['date'].dt.dayofweek


In [20]:
matches.head()


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,sh,pkatt,npxg/sh,gls,season,team,venue_code,opp_code,hour,day_code
3470,2018-08-10,20:00,Premier League,Matchweek 1,Fri,Home,W,2.0,1.0,Leicester City,...,7.0,1,0.10,2,2018,Manchester United,1,13,20,4
3584,2018-08-10,20:00,Premier League,Matchweek 1,Fri,Away,L,1.0,2.0,Manchester Utd,...,13.0,0,0.14,1,2018,Leicester City,0,17,20,4
3356,2018-08-11,15:00,Premier League,Matchweek 1,Sat,Away,W,3.0,0.0,Huddersfield,...,12.0,1,0.10,3,2018,Chelsea,0,11,15,5
3964,2018-08-11,15:00,Premier League,Matchweek 1,Sat,Home,L,0.0,2.0,Crystal Palace,...,15.0,0,0.05,0,2018,Fulham,1,8,15,5
3888,2018-08-11,15:00,Premier League,Matchweek 1,Sat,Away,L,0.0,2.0,Watford,...,6.0,0,0.05,0,2018,Brighton and Hove Albion,0,24,15,5


In [21]:
# wins, losses, or draws
matches['target'] = matches['result'].apply(lambda x: 1 if x == 'W' else (-1 if x == 'L' else 0))


In [22]:
matches.head()


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,pkatt,npxg/sh,gls,season,team,venue_code,opp_code,hour,day_code,target
3470,2018-08-10,20:00,Premier League,Matchweek 1,Fri,Home,W,2.0,1.0,Leicester City,...,1,0.10,2,2018,Manchester United,1,13,20,4,1
3584,2018-08-10,20:00,Premier League,Matchweek 1,Fri,Away,L,1.0,2.0,Manchester Utd,...,0,0.14,1,2018,Leicester City,0,17,20,4,-1
3356,2018-08-11,15:00,Premier League,Matchweek 1,Sat,Away,W,3.0,0.0,Huddersfield,...,1,0.10,3,2018,Chelsea,0,11,15,5,1
3964,2018-08-11,15:00,Premier League,Matchweek 1,Sat,Home,L,0.0,2.0,Crystal Palace,...,0,0.05,0,2018,Fulham,1,8,15,5,-1
3888,2018-08-11,15:00,Premier League,Matchweek 1,Sat,Away,L,0.0,2.0,Watford,...,0,0.05,0,2018,Brighton and Hove Albion,0,24,15,5,-1


In [23]:
matches['target'].value_counts(normalize=True)


target
 1    0.389604
-1    0.389604
 0    0.220792
Name: proportion, dtype: float64

In [24]:
#t initialize the model
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)


In [25]:
# train the model , careful with timeseries


X = matches[['venue_code', 'opp_code', 'hour', 'day_code']]
y = matches['target']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=1)


In [26]:
# print all the shapes of the data
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(3232, 4)
(3232,)
(808, 4)
(808,)


In [27]:
predictors = ['venue_code', 'opp_code', 'hour', 'day_code']


In [28]:
rf.fit(X_train, y_train)


RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [29]:
preds = rf.predict(X_test)


In [30]:
acc = accuracy_score(y_test, preds)


In [31]:
acc


0.40965346534653463

In [36]:
matches.columns


Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'referee', 'match report', 'notes', 'pk', 'g/sot', 'dist', 'sot%',
       'npxg', 'fk', 'np:g-xg', 'g-xg', 'sot', 'g/sh', 'sh', 'pkatt',
       'npxg/sh', 'gls', 'season', 'team', 'venue_code', 'opp_code', 'hour',
       'day_code', 'target'],
      dtype='object')

In [39]:
combined = pd.DataFrame({ 'actual': y_test, 'prediction': preds})


In [42]:
pd.crosstab(index=combined['actual'], columns=combined['prediction'], margins=True)


prediction,-1,0,1,All
actual,,,,
-1,154,45,115,314
0,97,14,85,196
1,95,40,163,298
All,346,99,363,808


We actually used accuracy as metric, but we want to optimize for predictions, we should optimize for precision

In [43]:
from sklearn.metrics import precision_score

# when we predict a win, how often is it a win
precision_score(y_test, preds, average=None)


array([0.44508671, 0.14141414, 0.44903581])

In [44]:
grouped_matches = matches.groupby('team')


In [45]:
group = grouped_matches.get_group('Manchester City')
group


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,pkatt,npxg/sh,gls,season,team,venue_code,opp_code,hour,day_code,target
3280,2018-08-12,16:00,Premier League,Matchweek 1,Sun,Away,W,2.0,0.0,Arsenal,...,0,0.10,2,2018,Manchester City,0,0,16,6,1
3281,2018-08-19,13:30,Premier League,Matchweek 2,Sun,Home,W,6.0,1.0,Huddersfield,...,0,0.14,5,2018,Manchester City,1,11,13,6,1
3282,2018-08-25,12:30,Premier League,Matchweek 3,Sat,Away,D,1.0,1.0,Wolves,...,0,0.09,1,2018,Manchester City,0,27,12,5,0
3283,2018-09-01,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Newcastle Utd,...,0,0.09,2,2018,Manchester City,1,18,17,5,1
3284,2018-09-15,15:00,Premier League,Matchweek 5,Sat,Home,W,3.0,0.0,Fulham,...,0,0.18,3,2018,Manchester City,1,10,15,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,2023-10-08,16:30,Premier League,Matchweek 8,Sun,Away,L,0.0,1.0,Arsenal,...,0,0.12,0,2023,Manchester City,0,0,16,6,-1
8,2023-10-21,15:00,Premier League,Matchweek 9,Sat,Home,W,2.0,1.0,Brighton,...,0,0.08,2,2023,Manchester City,1,4,15,5,1
9,2023-10-29,15:30,Premier League,Matchweek 10,Sun,Away,W,3.0,0.0,Manchester Utd,...,1,0.16,3,2023,Manchester City,0,17,15,6,1
10,2023-11-04,15:00,Premier League,Matchweek 11,Sat,Home,W,6.0,1.0,Bournemouth,...,0,0.09,6,2023,Manchester City,1,2,15,5,1


In [ ]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values('date')
    rolling_stats = group[cols].rolling(3, closed='left').mean() # does not include current match
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group


In [ ]:
cols = ["gf","ga","sh","sot","dist","fk","pk","pkatt"]
new_cols = [f"{c}_rolling" for c in cols]


In [ ]:
new_cols


['gf_rolling',
 'ga_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling']

In [ ]:
# testing for mancity
rolling_averages(group, cols, new_cols)


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
2551,2019-08-31,15:00,Premier League,Matchweek 4,Sat,Home,W,4.0,0.0,Brighton,...,5,1,3.333333,1.000000,20.333333,7.666667,15.633333,0.000000,0.333333,0.333333
2552,2019-09-14,17:30,Premier League,Matchweek 5,Sat,Away,L,2.0,3.0,Norwich City,...,5,0,3.000000,1.000000,21.000000,7.000000,15.200000,0.333333,0.000000,0.000000
2553,2019-09-21,15:00,Premier League,Matchweek 6,Sat,Home,W,8.0,0.0,Watford,...,5,1,3.000000,1.333333,19.333333,6.333333,15.100000,0.333333,0.000000,0.000000
2554,2019-09-28,17:30,Premier League,Matchweek 7,Sat,Away,W,3.0,1.0,Everton,...,5,1,4.666667,1.000000,22.333333,8.000000,15.133333,1.000000,0.333333,0.333333
2555,2019-10-06,14:00,Premier League,Matchweek 8,Sun,Home,L,0.0,2.0,Wolves,...,6,0,4.333333,1.333333,23.666667,8.666667,14.900000,1.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,2023-09-30,15:00,Premier League,Matchweek 7,Sat,Away,L,1.0,2.0,Wolves,...,5,0,3.333333,0.666667,14.000000,7.000000,16.133333,1.000000,0.333333,0.333333
7,2023-10-08,16:30,Premier League,Matchweek 8,Sun,Away,L,0.0,1.0,Arsenal,...,6,0,2.000000,1.000000,19.666667,8.333333,17.633333,1.333333,0.000000,0.000000
8,2023-10-21,15:00,Premier League,Matchweek 9,Sat,Home,W,2.0,1.0,Brighton,...,5,1,1.000000,1.000000,11.333333,4.000000,17.133333,1.000000,0.000000,0.000000
9,2023-10-29,15:30,Premier League,Matchweek 10,Sun,Away,W,3.0,0.0,Manchester Utd,...,6,1,1.000000,1.333333,12.333333,4.333333,17.666667,0.666667,0.000000,0.000000


In [81]:
# apply to all matches

matches_rolling = matches.groupby('team').apply(lambda x: rolling_averages(x, cols, new_cols))


In [82]:
matches_rolling.head()


date   time            comp        round  day venue result  \
team                                                                            
Arsenal 2779 2019-09-01  16:30  Premier League  Matchweek 4  Sun  Home      D   
        2780 2019-09-15  16:30  Premier League  Matchweek 5  Sun  Away      D   
        2781 2019-09-22  16:30  Premier League  Matchweek 6  Sun  Home      W   
        2782 2019-09-30  20:00  Premier League  Matchweek 7  Mon  Away      D   
        2783 2019-10-06  14:00  Premier League  Matchweek 8  Sun  Home      W   

               gf   ga        opponent  ...  day_code  target  gf_rolling  \
team                                    ...                                 
Arsenal 2779  2.0  2.0       Tottenham  ...         6       0    1.333333   
        2780  2.0  2.0         Watford  ...         6       0    1.666667   
        2781  3.0  2.0     Aston Villa  ...         6       1    1.666667   
        2782  1.0  1.0  Manchester Utd  ...         0       0    2.333333   
        2783  1.0  0.0     Bournemouth  ...         6       1    2.000000   

              ga_rolling sh_rolling sot_rolling dist_rolling fk_rolling  \
team                                                                      
Arsenal 2779    1.333333  10.666667    4.666667         17.2   0.333333   
        2780    2.000000  16.666667    6.666667         18.6   1.000000   
        2781    2.333333  14.000000    5.000000         19.3   1.000000   
        2782    2.000000  17.666667    5.666667         18.6   1.333333   
        2783    1.666667  12.333333    4.666667         18.1   1.000000   

              pk_rolling  pkatt_rolling  
team                                     
Arsenal 2779    0.000000       0.000000  
        2780    0.000000       0.000000  
        2781    0.000000       0.000000  
        2782    0.333333       0.333333  
        2783    0.333333       0.333333  

[5 rows x 48 columns]

In [83]:
matches_rolling = matches_rolling.droplevel('team')


In [85]:
matches_rolling


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
2779,2019-09-01,16:30,Premier League,Matchweek 4,Sun,Home,D,2.0,2.0,Tottenham,...,6,0,1.333333,1.333333,10.666667,4.666667,17.200000,0.333333,0.000000,0.000000
2780,2019-09-15,16:30,Premier League,Matchweek 5,Sun,Away,D,2.0,2.0,Watford,...,6,0,1.666667,2.000000,16.666667,6.666667,18.600000,1.000000,0.000000,0.000000
2781,2019-09-22,16:30,Premier League,Matchweek 6,Sun,Home,W,3.0,2.0,Aston Villa,...,6,1,1.666667,2.333333,14.000000,5.000000,19.300000,1.000000,0.000000,0.000000
2782,2019-09-30,20:00,Premier League,Matchweek 7,Mon,Away,D,1.0,1.0,Manchester Utd,...,0,0,2.333333,2.000000,17.666667,5.666667,18.600000,1.333333,0.333333,0.333333
2783,2019-10-06,14:00,Premier League,Matchweek 8,Sun,Home,W,1.0,0.0,Bournemouth,...,6,1,2.000000,1.666667,12.333333,4.666667,18.100000,1.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,2023-10-08,14:00,Premier League,Matchweek 8,Sun,Home,D,1.0,1.0,Aston Villa,...,6,0,1.333333,1.666667,5.666667,2.000000,12.366667,0.000000,0.000000,0.000000
122,2023-10-21,15:00,Premier League,Matchweek 9,Sat,Away,W,2.0,1.0,Bournemouth,...,5,1,1.333333,1.000000,4.666667,2.333333,11.466667,0.000000,0.000000,0.000000
123,2023-10-28,17:30,Premier League,Matchweek 10,Sat,Home,D,2.0,2.0,Newcastle Utd,...,5,0,1.666667,1.000000,10.666667,3.666667,14.533333,0.000000,0.000000,0.000000
124,2023-11-04,15:00,Premier League,Matchweek 11,Sat,Away,L,1.0,2.0,Sheffield Utd,...,5,0,1.666667,1.333333,13.333333,5.333333,16.466667,0.333333,0.000000,0.000000


In [86]:
matches_rolling.index = range(matches_rolling.shape[0])


In [87]:
matches_rolling


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2019-09-01,16:30,Premier League,Matchweek 4,Sun,Home,D,2.0,2.0,Tottenham,...,6,0,1.333333,1.333333,10.666667,4.666667,17.200000,0.333333,0.000000,0.000000
1,2019-09-15,16:30,Premier League,Matchweek 5,Sun,Away,D,2.0,2.0,Watford,...,6,0,1.666667,2.000000,16.666667,6.666667,18.600000,1.000000,0.000000,0.000000
2,2019-09-22,16:30,Premier League,Matchweek 6,Sun,Home,W,3.0,2.0,Aston Villa,...,6,1,1.666667,2.333333,14.000000,5.000000,19.300000,1.000000,0.000000,0.000000
3,2019-09-30,20:00,Premier League,Matchweek 7,Mon,Away,D,1.0,1.0,Manchester Utd,...,0,0,2.333333,2.000000,17.666667,5.666667,18.600000,1.333333,0.333333,0.333333
4,2019-10-06,14:00,Premier League,Matchweek 8,Sun,Home,W,1.0,0.0,Bournemouth,...,6,1,2.000000,1.666667,12.333333,4.666667,18.100000,1.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3184,2023-10-08,14:00,Premier League,Matchweek 8,Sun,Home,D,1.0,1.0,Aston Villa,...,6,0,1.333333,1.666667,5.666667,2.000000,12.366667,0.000000,0.000000,0.000000
3185,2023-10-21,15:00,Premier League,Matchweek 9,Sat,Away,W,2.0,1.0,Bournemouth,...,5,1,1.333333,1.000000,4.666667,2.333333,11.466667,0.000000,0.000000,0.000000
3186,2023-10-28,17:30,Premier League,Matchweek 10,Sat,Home,D,2.0,2.0,Newcastle Utd,...,5,0,1.666667,1.000000,10.666667,3.666667,14.533333,0.000000,0.000000,0.000000
3187,2023-11-04,15:00,Premier League,Matchweek 11,Sat,Away,L,1.0,2.0,Sheffield Utd,...,5,0,1.666667,1.333333,13.333333,5.333333,16.466667,0.333333,0.000000,0.000000


In [106]:
def make_predictions(data, predictors):
    train = data[data['date'] < '2023-01-01']
    test = data[data['date'] > '2023-01-01']
    rf.fit(train[predictors], train['target'])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame({'date': test['date'], 'team': test['team'], 'opponent': test['opponent'], 'result': test['result'], 'actual':test['target'], 'prediction':preds}, index=test.index)
    precision = precision_score(test['target'], preds)
    return combined, precision


In [107]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)


In [108]:
precision


0.5740740740740741

In [109]:
combined


,date,team,opponent,result,actual,prediction
127,2023-01-03,Arsenal,Newcastle Utd,D,0,1
128,2023-01-15,Arsenal,Tottenham,W,1,1
129,2023-01-22,Arsenal,Manchester Utd,W,1,0
130,2023-02-04,Arsenal,Everton,L,0,1
131,2023-02-11,Arsenal,Brentford,D,0,1
...,...,...,...,...,...,...
3184,2023-10-08,Wolverhampton Wanderers,Aston Villa,D,0,0
3185,2023-10-21,Wolverhampton Wanderers,Bournemouth,W,1,0
3186,2023-10-28,Wolverhampton Wanderers,Newcastle Utd,D,0,0
3187,2023-11-04,Wolverhampton Wanderers,Sheffield Utd,L,0,1


In [122]:
%pip install pprint


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pprint (from versions: none)
ERROR: No matching distribution found for pprint


In [125]:
import pprint as pp

pp.pprint(set(combined['team'].tolist()))


{'Arsenal',
 'Aston Villa',
 'Bournemouth',
 'Brentford',
 'Brighton and Hove Albion',
 'Burnley',
 'Chelsea',
 'Crystal Palace',
 'Everton',
 'Fulham',
 'Leeds United',
 'Leicester City',
 'Liverpool',
 'Luton Town',
 'Manchester City',
 'Manchester United',
 'Newcastle United',
 'Nottingham Forest',
 'Sheffield United',
 'Southampton',
 'Tottenham Hotspur',
 'West Ham United',
 'Wolverhampton Wanderers'}


In [126]:
pp.pprint(set(combined['opponent'].tolist()))


{'Arsenal',
 'Aston Villa',
 'Bournemouth',
 'Brentford',
 'Brighton',
 'Burnley',
 'Chelsea',
 'Crystal Palace',
 'Everton',
 'Fulham',
 'Leeds United',
 'Leicester City',
 'Liverpool',
 'Luton Town',
 'Manchester City',
 'Manchester Utd',
 'Newcastle Utd',
 "Nott'ham Forest",
 'Sheffield Utd',
 'Southampton',
 'Tottenham',
 'West Ham',
 'Wolves'}


In [132]:
set_home = set(combined['team'].tolist())
set_away = set(combined['opponent'].tolist())

difference = set_home.symmetric_difference(set_away)


In [130]:
difference


{'Brighton and Hove Albion',
 'Manchester United',
 'Newcastle United',
 'Nottingham Forest',
 'Sheffield United',
 'Tottenham Hotspur',
 'West Ham United',
 'Wolverhampton Wanderers'}

In [134]:
# normalize the names of teams home and away

class MissingDict(dict):
    __missing__ = lambda self, key: key


map_values = {
   'Brighton and Hove Albion':'Brighton',
   'Manchester United': 'Manchester Utd',
   'Newcastle United': 'Newcastle Utd',
   'Nottingham Forest': "Nott'ham Forest",
   'Sheffield United': 'Sheffield Utd',
   'Tottenham Hotspur': 'Tottenham',
   'West Ham United': 'West Ham',
   'Wolverhampton Wanderers': 'Wolves'
}
mapping = MissingDict(**map_values)


In [136]:
mapping['Arsenal']


'Arsenal'

In [137]:
mapping['Brighton and Hove Albion']


'Brighton'

In [138]:
combined['new_team'] = combined['team'].map(mapping)


In [139]:
combined


,date,team,opponent,result,actual,prediction,new_team
127,2023-01-03,Arsenal,Newcastle Utd,D,0,1,Arsenal
128,2023-01-15,Arsenal,Tottenham,W,1,1,Arsenal
129,2023-01-22,Arsenal,Manchester Utd,W,1,0,Arsenal
130,2023-02-04,Arsenal,Everton,L,0,1,Arsenal
131,2023-02-11,Arsenal,Brentford,D,0,1,Arsenal
...,...,...,...,...,...,...,...
3184,2023-10-08,Wolverhampton Wanderers,Aston Villa,D,0,0,Wolves
3185,2023-10-21,Wolverhampton Wanderers,Bournemouth,W,1,0,Wolves
3186,2023-10-28,Wolverhampton Wanderers,Newcastle Utd,D,0,0,Wolves
3187,2023-11-04,Wolverhampton Wanderers,Sheffield Utd,L,0,1,Wolves


In [140]:
# checking both sides of predictions
merged = combined.merge(combined, left_on=['date', 'new_team'], right_on=['date', 'opponent'])


In [141]:
merged


,date,team_x,opponent_x,result_x,actual_x,prediction_x,new_team_x,team_y,opponent_y,result_y,actual_y,prediction_y,new_team_y
0,2023-01-03,Arsenal,Newcastle Utd,D,0,1,Arsenal,Newcastle United,Arsenal,D,0,0,Newcastle Utd
1,2023-01-15,Arsenal,Tottenham,W,1,1,Arsenal,Tottenham Hotspur,Arsenal,L,0,0,Tottenham
2,2023-01-22,Arsenal,Manchester Utd,W,1,0,Arsenal,Manchester United,Arsenal,L,0,1,Manchester Utd
3,2023-02-04,Arsenal,Everton,L,0,1,Arsenal,Everton,Arsenal,W,1,0,Everton
4,2023-02-11,Arsenal,Brentford,D,0,1,Arsenal,Brentford,Arsenal,D,0,0,Brentford
...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,2023-10-08,Wolverhampton Wanderers,Aston Villa,D,0,0,Wolves,Aston Villa,Wolves,D,0,1,Aston Villa
648,2023-10-21,Wolverhampton Wanderers,Bournemouth,W,1,0,Wolves,Bournemouth,Wolves,L,0,0,Bournemouth
649,2023-10-28,Wolverhampton Wanderers,Newcastle Utd,D,0,0,Wolves,Newcastle United,Wolves,D,0,1,Newcastle Utd
650,2023-11-04,Wolverhampton Wanderers,Sheffield Utd,L,0,1,Wolves,Sheffield United,Wolves,W,1,1,Sheffield Utd


In [143]:
# checking model when it predicted for home team to win and away team to lose

merged[(merged["prediction_x"] == 1) & (merged["prediction_y"] == 0)]["actual_x"].value_counts()


actual_x
1    85
0    57
Name: count, dtype: int64

In [146]:
# accuracy of model
85/(85+57)


0.5985915492957746

# To do possibilities

 - add more seasons
 - use more preditors
 - use other ML algoritm
 - more championships
 - time series analysis - seasonal effects ?
 - add odds to data from football data
 - back testing strategies with odds